In [13]:
#Task 2 (a)

In [5]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

# Demand at each location
demand = {
    "Santa Ana": 22418,
    "El Paso": 6800,
    "Pendleton": 80290,
    "Houston": 100447,
    "Kansas City": 24570,
    "Los Angeles": 64761,
    "Glendale": 33689,
    "Jacksonville": 68486,
    "Little Rock": 148586,
    "Bridgeport": 111475,
    "Sacramento": 112000
}

# Supply limits
supply_capacity = {
    "Cincinnati": 500000,
    "Oakland": 500000  
}

# Production costs
production_cost = {
    "Cincinnati": 1.20,
    "Oakland": 1.65
}

freight_cost = {
    ("Cincinnati", "Santa Ana"): None,
    ("Cincinnati", "El Paso"): 0.84,
    ("Cincinnati", "Pendleton"): 0.83,
    ("Cincinnati", "Houston"): 0.45,
    ("Cincinnati", "Kansas City"): 0.36,
    ("Cincinnati", "Los Angeles"): None,
    ("Cincinnati", "Glendale"): None,
    ("Cincinnati", "Jacksonville"): 0.34,
    ("Cincinnati", "Little Rock"): 0.34,
    ("Cincinnati", "Bridgeport"): 0.34,
    ("Cincinnati", "Sacramento"): None,
    
    ("Oakland", "Santa Ana"): 0.22,
    ("Oakland", "El Paso"): 0.74,
    ("Oakland", "Pendleton"): 0.49,
    ("Oakland", "Houston"): None,
    ("Oakland", "Kansas City"): None,
    ("Oakland", "Los Angeles"): 0.22,
    ("Oakland", "Glendale"): 0.22,
    ("Oakland", "Jacksonville"): None,
    ("Oakland", "Little Rock"): None,
    ("Oakland", "Bridgeport"): None,
    ("Oakland", "Sacramento"): 0.15
}

model = LpProblem("Transportation_Problem", LpMinimize)

x = {}
for s in supply_capacity.keys():
    for d in demand.keys():
        if freight_cost[(s, d)] is not None:
            x[(s, d)] = LpVariable(f"x_{s}_{d}", lowBound=0)

model += lpSum((production_cost[s] + freight_cost[(s, d)]) * x[(s, d)]
               for s in supply_capacity.keys()
               for d in demand.keys()
               if (s, d) in x)


for d in demand:
    model += lpSum(x[(s, d)] for s in supply_capacity.keys() if (s, d) in x) == demand[d], f"Demand_{d}"

for s in supply_capacity:
    model += lpSum(x[(s, d)] for d in demand if (s, d) in x) <= supply_capacity[s], f"Supply_{s}"

model.solve()

print("Status:", LpStatus[model.status])

if LpStatus[model.status] == "Optimal":
    total_cost = value(model.objective)
    print("Total Minimum Cost: $", round(total_cost, 2))
    print("\nOptimal Shipping Plan:")
    for s,dest in x:
        qty = x[(s, dest)].varValue
        if qty > 0:
            print(f"{s} -> {dest}: {qty} gallons")

for name, constraint in model.constraints.items():
    print(f"{name} shadow price = {constraint.pi}")

for var in model.variables():
    print(f"{var.name} reduced cost = {var.dj}")


Status: Optimal
Total Minimum Cost: $ 1318984.93

Optimal Shipping Plan:
Cincinnati -> El Paso: 6800.0 gallons
Cincinnati -> Pendleton: 39636.0 gallons
Cincinnati -> Houston: 100447.0 gallons
Cincinnati -> Kansas City: 24570.0 gallons
Cincinnati -> Jacksonville: 68486.0 gallons
Cincinnati -> Little Rock: 148586.0 gallons
Cincinnati -> Bridgeport: 111475.0 gallons
Oakland -> Santa Ana: 22418.0 gallons
Oakland -> Pendleton: 40654.0 gallons
Oakland -> Los Angeles: 64761.0 gallons
Oakland -> Glendale: 33689.0 gallons
Oakland -> Sacramento: 112000.0 gallons
Demand_Santa_Ana shadow price = 1.87
Demand_El_Paso shadow price = 2.15
Demand_Pendleton shadow price = 2.14
Demand_Houston shadow price = 1.76
Demand_Kansas_City shadow price = 1.67
Demand_Los_Angeles shadow price = 1.87
Demand_Glendale shadow price = 1.87
Demand_Jacksonville shadow price = 1.65
Demand_Little_Rock shadow price = 1.65
Demand_Bridgeport shadow price = 1.65
Demand_Sacramento shadow price = 1.8
Supply_Cincinnati shadow pric

In [15]:
#Task 2 (e)

In [25]:
import pulp

def solve_transportation_problem(production_costs, shipping_costs, demands, max_cincinnati):
    sources = ["Cincinnati", "Oakland"]
    
    destinations = list(demands.keys())
    
    x = pulp.LpVariable.dicts("Ship", (sources, destinations), lowBound=0, cat=pulp.LpContinuous)
    
    # Initialize the LP problem
    prob = pulp.LpProblem("TransportationProblem", pulp.LpMinimize)
    
    prob += pulp.lpSum((production_costs[s] + shipping_costs[s][d]) * x[s][d] for s in sources for d in destinations)
    
    # Demand constraints: Each destination's demand must be met
    for d in destinations:
        prob += pulp.lpSum(x[s][d] for s in sources) == demands[d], f"Demand_{d}"
    
    prob += pulp.lpSum(x["Cincinnati"][d] for d in destinations) <= max_cincinnati, "Cincinnati_Capacity"
    

    prob += pulp.lpSum(x["Oakland"][d] for d in destinations) <= 500000, "Oakland_Capacity"
    
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    
    # Output the results
    status = pulp.LpStatus[prob.status]
    total_cost = pulp.value(prob.objective)
    
    solution = {s: {} for s in sources}
    for s in sources:
        for d in destinations:
            solution[s][d] = x[s][d].varValue
    
    return status, total_cost, solution


demands = {
    "Santa Ana": 22418,
    "Glendale": 33689,
    "El Paso": 6800,
    "Jacksonville": 68486,
    "Pendleton": 80290,
    "Little Rock": 148586,
    "Houston": 100447,
    "Bridgeport": 111475,
    "Kansas City": 24570,
    "Sacramento": 112000,
    "Los Angeles": 64761
}

# Production costs
production_costs = {
    "Cincinnati": 1.20,
    "Oakland": 1.65
}


large_cost = 999999

shipping_costs = {
    "Cincinnati": {
        "Santa Ana": large_cost,  
        "Glendale": large_cost,  
        "El Paso": 0.84,
        "Jacksonville": 0.34,
        "Pendleton": 0.83,
        "Little Rock": 0.34,
        "Houston": 0.45,
        "Bridgeport": 0.34,
        "Kansas City": 0.36,
        "Sacramento": large_cost, 
        "Los Angeles": large_cost 
    },
    "Oakland": {
        "Santa Ana": 0.22,
        "Glendale": 0.22,
        "El Paso": 0.74,
        "Jacksonville": large_cost, 
        "Pendleton": 0.49,
        "Little Rock": large_cost, 
        "Houston": large_cost,   
        "Bridgeport": large_cost, 
        "Kansas City": large_cost, 
        "Sacramento": 0.15,
        "Los Angeles": 0.22
    }
}

# Maximum Cincinnati production
max_cincinnati = 500000


status, total_cost, solution = solve_transportation_problem(production_costs, shipping_costs, demands, max_cincinnati)

print("Status:", status)
print("Total Cost:", total_cost)
for s in solution:
    for d in solution[s]:
        if solution[s][d] > 0:
            print(f"Ship {solution[s][d]} gallons from {s} to {d}")


def adjust_costs(production_costs, shipping_costs, prod_factor=1.0, ship_factor=1.0):
    new_production = {s: production_costs[s]*prod_factor for s in production_costs}
    new_shipping = {
        s: {d: shipping_costs[s][d]*ship_factor if shipping_costs[s][d] < large_cost else large_cost 
            for d in shipping_costs[s]} 
        for s in shipping_costs
    }
    return new_production, new_shipping

# Example: Increase production costs and shipping rates by 10%
prod_up_10, ship_up_10 = adjust_costs(production_costs, shipping_costs, prod_factor=1.1, ship_factor=1.0)
status_up, total_cost_up, solution_up = solve_transportation_problem(prod_up_10, ship_up_10, demands, max_cincinnati)
print("\n--- After 10% Increase in Production Costs ---")
print("Status:", status_up)
print("Total Cost:", total_cost_up)
for s in solution:
    for d in solution[s]:
        if solution[s][d] > 0:
            print(f"Ship {solution[s][d]} gallons from {s} to {d}")

prod_up_10, ship_up_10 = adjust_costs(production_costs, shipping_costs, prod_factor=1.0, ship_factor=1.1)
status_up, total_cost_up, solution_up = solve_transportation_problem(prod_up_10, ship_up_10, demands, max_cincinnati)
print("\n--- After 10% Increase in Shipping Rates ---")
print("Status:", status_up)
print("Total Cost:", total_cost_up)
for s in solution:
    for d in solution[s]:
        if solution[s][d] > 0:
            print(f"Ship {solution[s][d]} gallons from {s} to {d}")


# Example: Decrease production costs and shipping rates by 10%
prod_down_10, ship_down_10 = adjust_costs(production_costs, shipping_costs, prod_factor=0.9, ship_factor=1)
status_down, total_cost_down, solution_down = solve_transportation_problem(prod_down_10, ship_down_10, demands, max_cincinnati)
print("\n--- After 10% Decrease in Production Costs ---")
print("Status:", status_down)
print("Total Cost:", total_cost_down)
for s in solution:
    for d in solution[s]:
        if solution[s][d] > 0:
            print(f"Ship {solution[s][d]} gallons from {s} to {d}")

prod_down_10, ship_down_10 = adjust_costs(production_costs, shipping_costs, prod_factor=1, ship_factor=0.9)
status_down, total_cost_down, solution_down = solve_transportation_problem(prod_down_10, ship_down_10, demands, max_cincinnati)
print("\n--- After 10% Decrease in Shipping Rates ---")
print("Status:", status_down)
print("Total Cost:", total_cost_down)

for s in solution:
    for d in solution[s]:
        if solution[s][d] > 0:
            print(f"Ship {solution[s][d]} gallons from {s} to {d}")


Status: Optimal
Total Cost: 1318984.93
Ship 6800.0 gallons from Cincinnati to El Paso
Ship 68486.0 gallons from Cincinnati to Jacksonville
Ship 39636.0 gallons from Cincinnati to Pendleton
Ship 148586.0 gallons from Cincinnati to Little Rock
Ship 100447.0 gallons from Cincinnati to Houston
Ship 111475.0 gallons from Cincinnati to Bridgeport
Ship 24570.0 gallons from Cincinnati to Kansas City
Ship 22418.0 gallons from Oakland to Santa Ana
Ship 33689.0 gallons from Oakland to Glendale
Ship 40654.0 gallons from Oakland to Pendleton
Ship 112000.0 gallons from Oakland to Sacramento
Ship 64761.0 gallons from Oakland to Los Angeles

--- After 10% Increase in Production Costs ---
Status: Optimal
Total Cost: 1424116.06
Ship 6800.0 gallons from Cincinnati to El Paso
Ship 68486.0 gallons from Cincinnati to Jacksonville
Ship 39636.0 gallons from Cincinnati to Pendleton
Ship 148586.0 gallons from Cincinnati to Little Rock
Ship 100447.0 gallons from Cincinnati to Houston
Ship 111475.0 gallons from C